In [ ]:
# install env
!pip3 install torch
!pip3 install torchvision

!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda=9.0.176-1

In [ ]:
# test env
import torch
!/opt/bin/nvidia-smi

In [ ]:
# mount google drive to env

from google.colab import drive
import os
drive.mount('/content/drive')
data_dir = "/content/drive/My Drive/colab/data/"

In [1]:
# tcn

import torch
import torch.nn as nn
from torch.nn.utils import weight_norm


class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size, padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

print("TCN component initialized")


TCN component initialized


In [2]:
# model

from torch import nn
import torch.nn.functional as F


class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCN, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size, dropout=dropout)
        self.linear = nn.Linear(num_channels[-1], output_size)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        # x needs to have dimension (N, C, L) in order to be passed into CNN
        output = self.tcn(x.transpose(1, 2)).transpose(1, 2)
        output = self.linear(output).float()
        return self.sig(output)


print("TCN model initialized")


TCN model initialized


In [9]:
# data_generator

import torch
import numpy as np
from tqdm import tqdm

nan_mode = 1 # method to manage NaN value
data_dir = "../data/training/"


def read_psv(f, X, Y):
  lines = f.readlines()
  m = np.zeros((len(lines)-1, 41), dtype=np.float32)
  for i in range(1, len(lines)):
    line = lines[i][:-1].split('|')
    line = np.array(line, dtype=np.float32)
    X[i-1] = line[:-1]
    Y[i-1] = line[-1]


def nan_process(m, alg=0):
  if alg == 0:
    # fill NaN value with 0 
    m[np.isnan(m)] = 0
  elif alg == 1:
    # TODO: fill NaN value with average
    pass
  return m


def data_generator(dataset='B', nan_alg=0):
  '''
  读入数据并分割为训练集、验证集、测试集，默认使用setB
  
  args:
    dataset: 使用的数据集         'A'为竞赛数据集A，'B'为竞赛数据集B，'M'为MIMIC-III数据集
    nan_alg: 处理空值使用的算法    0为填充0，1为填充平均值

  note:
    setA数据文件索引范围1~20643，19000之后有序号缺失，共计20337个病人的记录，最长时间序列长度336
      导出矩阵尺寸为 X:(20337, 336, 40) Y:(20337, 336)
    setB数据文件索引范围100001~120000，共计20000个病人的记录，最长时间序列长度336
      导出矩阵尺寸为 X:(20000, 336, 40) Y:(20000, 336)
  '''

  file_name_pattern = 'p{:0=6}.psv'
  if dataset == 'A':
    file_index = list(range(1, 20644))
  elif dataset == 'B':
    file_index = list(range(100001, 102001))
  elif dataset == 'M':
    pass

  np.random.seed(1234888)
  np.random.shuffle(file_index)

  split_ratio = np.array([0.7, 0.15, 0.15]) # train_set 0.7, valid_set 0.15, test_set 0.15
  split_len = np.round(split_ratio * len(file_index)).astype(np.int16) 
  dataset_index = [
    file_index[:split_len[0]],                            # train_set_index
    file_index[split_len[0]:split_len[0] + split_len[1]], # valid_set_index
    file_index[split_len[0] + split_len[1]:]             # test_set_index
  ]
  
  datas = [
    [np.zeros((split_len[0], 336, 40)), np.zeros((split_len[0], 336))], # train_set
    [np.zeros((split_len[1], 336, 40)), np.zeros((split_len[1], 336))], # valid_set
    [np.zeros((split_len[2], 336, 40)), np.zeros((split_len[2], 336))], # test_set
  ]

  # read file
  for i in range(3):
    for j in tqdm(dataset_index[i]):
      file_name = file_name_pattern.format(j + 1)
      try:
        datafile = open(data_dir + file_name)
        read_psv(datafile, datas[i][0][j], datas[i][1][j])
      except:
        pass
      finally:
        datafile.close()
    datas[i][0] = nan_process(datas[i][0], nan_alg)
    datas[i][1] = nan_process(datas[i][1], nan_alg)
  
  for data in datas:
    data[1] = torch.Tensor(data[1])
    data[0] = torch.Tensor(data[0])

  return datas[0], datas[1], datas[2]

print("data generator initialized")

data generator initialized


In [10]:
# sepsis.py

import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np


if not torch.cuda.is_available():
    panic("CUDA is not available")

# ------------------ arg -------------------------

dropout = 0.25
clip = 0.2
optim = 'Adam'
kernel_size = 5
log_interval = 100
lr = 1e-3
epochs = 100

nhid = 150
levels = 4
input_size = 88
seed = 6783
model_version = 1

# -----------------------------------------------

# data
train, valid, test = data_generator()

# model
model = TCN(input_size, 1, n_channels, kernel_size, dropout=_dropout)
model.cuda()

# Set the random seed manually for reproducibility.
torch.manual_seed(seed)
n_channels = [nhid] * levels
criterion = nn.CrossEntropyLoss()
optimizer = getattr(torch.optim, optim)(model.parameters(), lr=lr)

def evaluate(X_data, name='Eval'):
    model.eval()
    eval_idx_list = np.arange(len(X_data), dtype="int32")
    total_loss = 0.0
    count = 0
    with torch.no_grad():
        for idx in eval_idx_list:
            data_line = X_data[idx]
            x, y = Variable(data_line[:-1]), Variable(data_line[1:])
            if _cuda:
                x, y = x.cuda(), y.cuda()
            output = model(x.unsqueeze(0)).squeeze(0)
            loss = -torch.trace(torch.matmul(y, torch.log(output).float().t()) +
                                torch.matmul((1-y), torch.log(1-output).float().t()))
            total_loss += loss.item()
            count += output.size(0)
        eval_loss = total_loss / count
        print(name + " loss: {:.5f}".format(eval_loss))
        return eval_loss


def train(ep):
    model.train()
    total_loss = 0
    count = 0
    train_idx_list = np.arange(len(X_train), dtype="int32")
    np.random.shuffle(train_idx_list)
    for idx in train_idx_list:
        data_line = X_train[idx]
        x, y = Variable(data_line[:-1]), Variable(data_line[1:])
        if _cuda:
            x, y = x.cuda(), y.cuda()

        optimizer.zero_grad()
        output = model(x.unsqueeze(0)).squeeze(0)
        loss = -torch.trace(torch.matmul(y, torch.log(output).float().t()) +
                            torch.matmul((1 - y), torch.log(1 - output).float().t()))
        total_loss += loss.item()
        count += output.size(0)

        if clip > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        loss.backward()
        optimizer.step()
        if idx > 0 and idx % log_interval == 0:
            cur_loss = total_loss / count
            print("Epoch {:2d} | lr {:.5f} | loss {:.5f}".format(ep, lr, cur_loss))
            total_loss = 0.0
            count = 0

best_vloss = 1e8
vloss_list = []
model_name = "sepsis_predict_{0}.pt".format(model_version)
for ep in range(1, epochs+1):
    train(ep)
    vloss = evaluate(valid, name='Validation')
    tloss = evaluate(test, name='Test')
    if vloss < best_vloss:
        with open(model_name, "wb") as f:
            torch.save(model, f)
            print("Saved model!\n")
        best_vloss = vloss
    if ep > 10 and vloss > max(vloss_list[-3:]):
        lr /= 10
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

    vloss_list.append(vloss)

print('-' * 89)
model = torch.load(open(model_name, "rb"))
tloss = evaluate(test)

  0%|          | 0/1400 [00:00<?, ?it/s]


UnboundLocalError: local variable 'datafile' referenced before assignment

In [6]:
import numpy as np
import torch

# split_ratio = np.array([0.7, 0.15, 0.15]) # train_set 0.7, valid_set 0.15, test_set 0.15
# split_len = np.round(split_ratio * 20001).astype(np.int16) 

# print(split_len[0])

# print(split_ratio)
# print(split_len)

# torch.Tensor(np.random.rand(2,2))

a = np.random.rand(2, 2, 3)
b = np.zeros((2, 3))
a0 = a[0]
a0[1,2] = 0
a0[0] = b[0, :]
print(a0)
print(a)


[[0.         0.         0.        ]
 [0.62074491 0.44682882 0.        ]]
[[[0.         0.         0.        ]
  [0.62074491 0.44682882 0.        ]]

 [[0.68417997 0.29076988 0.75393449]
  [0.80377299 0.06026075 0.95169849]]]
